In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

This part is only necessary if you are working on Google Colab!


In [46]:
import os

# save the model on Google Drive, link Google drive to this notebook
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
!pwd

/content


In [48]:
#!ls data/X-Ray-Scanner-Dataset
!ls /content/gdrive/'My Drive'/'Colab Notebooks'/WCAT

WCAT


In [0]:
#data_dir = 'data/X-Ray-Scanner-Dataset'   # path version for working locally
data_dir = "/content/gdrive/My Drive/Colab Notebooks/WCAT/WCAT/data/X-Ray-Scanner-Dataset"    # path version for working on Google Cola

In [0]:
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.Grayscale(num_output_channels=3),   # use this if your images are just greyscale and the model requires RGB
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  ## Not the correct values!
                                       ])

test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.Grayscale(num_output_channels=3),
                                      transforms.ToTensor(),
                                     ])

In [0]:
def load_split_train_test(datadir, valid_size = .2):
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)    
    
    # define the data split
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    
    
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64, drop_last=True)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64, drop_last=True)
    return trainloader, testloader

In [0]:
trainloader, testloader = load_split_train_test(data_dir, .2)

In [53]:
print(trainloader.dataset.classes)

['Dangerous', 'Not Dangerous']


In [54]:
    len(testloader)

1

Check if training on cuda is available

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [56]:
print (device)

cpu


Load the pretrained model from pytorch's model library

In [57]:
model = models.resnet50(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:00<00:00, 140157899.10it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
for param in model.parameters():
    param.requires_grad = False
        

Replace the fully connected Layer with our classifier

In [63]:
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 2), ### was 512, 10
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
epochs = 10
steps = 1
running_loss = 0
print_every = 1

In [65]:

train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. Steps {steps}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0


Epoch 1/10.. Steps 2.. Train loss: 0.760.. Test loss: 0.895.. Test accuracy: 0.719
Epoch 1/10.. Steps 3.. Train loss: 0.610.. Test loss: 2.179.. Test accuracy: 0.469
Epoch 1/10.. Steps 4.. Train loss: 2.173.. Test loss: 0.193.. Test accuracy: 0.938
Epoch 1/10.. Steps 5.. Train loss: 0.117.. Test loss: 0.687.. Test accuracy: 0.875
Epoch 2/10.. Steps 6.. Train loss: 0.168.. Test loss: 0.992.. Test accuracy: 0.859
Epoch 2/10.. Steps 7.. Train loss: 0.447.. Test loss: 0.786.. Test accuracy: 0.906
Epoch 2/10.. Steps 8.. Train loss: 0.326.. Test loss: 0.334.. Test accuracy: 0.906
Epoch 2/10.. Steps 9.. Train loss: 0.012.. Test loss: 0.250.. Test accuracy: 0.938
Epoch 3/10.. Steps 10.. Train loss: 0.000.. Test loss: 0.157.. Test accuracy: 0.953
Epoch 3/10.. Steps 11.. Train loss: 0.026.. Test loss: 0.177.. Test accuracy: 0.984
Epoch 3/10.. Steps 12.. Train loss: 0.001.. Test loss: 0.274.. Test accuracy: 0.906
Epoch 3/10.. Steps 13.. Train loss: 0.123.. Test loss: 0.236.. Test accuracy: 0.969


In [0]:
save_path = "/content/gdrive/My Drive/Colab Notebooks/WCAT/WCAT/wcat_model.pth"

In [0]:
 torch.save(model, save_path)